In [1]:
# pip install pandas numpy keras scikit-learn
import pandas as pd 
import numpy as np 
import warnings 
warnings.filterwarnings('ignore')


# **A. Build a baseline model** 

Use the Keras library to build a neural network with the following: 
1. One hidden layer of 10 nodes, and a ReLU activation function 
2. User the `adam` optimizer and `mean squared error` as the loss function. 

Steps: 
1. Randomly split the data into training and test sets by holding 30% of the data for testing. Use `train_test_split` from Scikit-learn 
2. Train the model on the training data using `50 epochs` 
3. Evaluate the model on the test data and compute the mean squared error between the predicted concrete strength and actual concrete strength. Use `mean_squared_error` from Scikit-learn
4. Repeat 1-3, 50 times and create a list of 50 MSEs. 
5. Report the mean and standard deviation of the mean squared errors. 

In [2]:
concrete_data = pd.read_csv('https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0101EN/labs/data/concrete_data.csv')
concrete_data.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


In [3]:
concrete_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1030 entries, 0 to 1029
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Cement              1030 non-null   float64
 1   Blast Furnace Slag  1030 non-null   float64
 2   Fly Ash             1030 non-null   float64
 3   Water               1030 non-null   float64
 4   Superplasticizer    1030 non-null   float64
 5   Coarse Aggregate    1030 non-null   float64
 6   Fine Aggregate      1030 non-null   float64
 7   Age                 1030 non-null   int64  
 8   Strength            1030 non-null   float64
dtypes: float64(8), int64(1)
memory usage: 72.6 KB


In [4]:
concrete_data.isnull().sum()

Cement                0
Blast Furnace Slag    0
Fly Ash               0
Water                 0
Superplasticizer      0
Coarse Aggregate      0
Fine Aggregate        0
Age                   0
Strength              0
dtype: int64

In [6]:
from sklearn.model_selection import train_test_split

y = concrete_data['Strength']
X = concrete_data[concrete_data.columns[concrete_data.columns != 'Strength']] 
test_size = 0.3
# Split 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size)  
n_cols = X_train.shape[1] 
n_cols


8

In [8]:
from keras import Sequential
from keras.layers import Dense 
def regression_model(n_cols: int): 
    model = Sequential() 
    model.add(Dense(10, activation='relu', input_shape=(n_cols,))) 
    model.add(Dense(1))

    # Compile 
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model 

model = regression_model(n_cols) 
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_2 (Dense)                 │ (None, 10)             │            90 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            11 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 101 (404.00 B)

 Trainable params: 101 (404.00 B)

 Non-trainable params: 0 (0.00 B)

In [9]:
from sklearn.metrics import mean_squared_error 

def run_model():  
    # 1. Split 
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size)   
    n_cols = X_train.shape[1] 
    model = regression_model(n_cols) 
    # 2. Train 
    model.fit(X_train, y_train, epochs=50, verbose=0) 
    # 3. Evaluate 
    y_predicted = model.predict(X_test)  
    mse = mean_squared_error(y_test, y_predicted) 
    
    return mse 
    

In [10]:
from tqdm import tqdm 
# 4. Repeat 50 times 
mse_values = [] 
for _ in tqdm(range(50), desc="Training model"): 
    mse_values.append(run_model()) 

mse_mean = np.mean(mse_values)
mse_std_dev = np.std(mse_values) 
print(f"MSE Mean: {mse_mean}, MSE Std. Dev: {mse_std_dev}")

Training model:   0%|          | 0/50 [00:00<?, ?it/s]

10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 


Training model:   2%|▏         | 1/50 [00:03<02:28,  3.03s/it]

10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 


Training model:   4%|▍         | 2/50 [00:05<02:13,  2.77s/it]

...

10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 


Training model:  98%|█████████▊| 49/50 [02:12<00:03,  3.00s/it]

10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 


Training model: 100%|██████████| 50/50 [02:15<00:00,  2.71s/it]

MSE Mean: 312.2150544126538, MSE Std. Dev: 375.98637038907026


# **A. MSE Mean: 312.215, MSE Standard Deviation: 375.986**

# **B. Normalize the data** 

Repeat A but use a normalized version of the data. 

How does the mean of the mean squared errors compare to that of step A? 

In [11]:
def normalize(data: any): 
    mean = data.mean() 
    std = data.std() 
    return (data - mean) / std 

def run_model():  
    # 1. Split 
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size)   
    n_cols = X_train.shape[1] 
    model = regression_model(n_cols) 
    # 1.1 Normalize 
    X_train_normalized = normalize(X_train)
    X_test_normalized = normalize(X_test)
    # 2. Train 
    model.fit(X_train_normalized, y_train, epochs=50, verbose=0) 
    # 3. Evaluate 
    y_predicted = model.predict(X_test_normalized)  
    mse = mean_squared_error(y_test, y_predicted) 
    
    return mse 

In [12]:
from tqdm import tqdm 
# 4. Repeat 50 times 
mse_values = [] 
for _ in tqdm(range(50), desc="Training model"): 
    mse_values.append(run_model()) 

mse_mean = np.mean(mse_values)
mse_std_dev = np.std(mse_values) 
print(f"MSE Mean: {mse_mean}, MSE Std. Dev: {mse_std_dev}")

Training model:   0%|          | 0/50 [00:00<?, ?it/s]

10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 


Training model:   2%|▏         | 1/50 [00:02<02:11,  2.69s/it]

10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 


Training model:   4%|▍         | 2/50 [00:05<02:08,  2.69s/it]

...

10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 


Training model:  98%|█████████▊| 49/50 [02:13<00:02,  2.74s/it]

10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 


Training model: 100%|██████████| 50/50 [02:16<00:00,  2.73s/it]

MSE Mean: 349.6364545362458, MSE Std. Dev: 109.88859881641301


# **B. MSE Mean: 349.636 MSE Std. Dev: 109.885** 

Observations: 
1. After normalizing, the mean of the MSEs remained relatively close to the mean prior to normalizing. 
2. Key difference lies in Standard Deviation: After normalizing, results became more consistent, with a standard deviation of 109.888, while a non-normalized run generated a standard deviation of 375.986

# **C. Repeat part B but use 100 epochs** 

In [15]:

def normalize(data: any): 
    mean = data.mean() 
    std = data.std() 
    return (data - mean) / std 

def run_model():  
    # 1. Split 
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size)   
    n_cols = X_train.shape[1] 
    model = regression_model(n_cols) 
    # 1.1 Normalize 
    X_train_normalized = normalize(X_train)
    X_test_normalized = normalize(X_test)
    # 2. Train 
    model.fit(X_train_normalized, y_train, epochs=100, verbose=0) 
    # 3. Evaluate 
    y_predicted = model.predict(X_test_normalized)  
    mse = mean_squared_error(y_test, y_predicted) 
    
    return mse 

In [16]:
from tqdm import tqdm 
# 4. Repeat 50 times 
mse_values = [] 
for _ in tqdm(range(50), desc="Training model"): 
    mse_values.append(run_model()) 

mse_mean = np.mean(mse_values)
mse_std_dev = np.std(mse_values) 
print(f"MSE Mean: {mse_mean}, MSE Std. Dev: {mse_std_dev}")

Training model:   0%|          | 0/50 [00:00<?, ?it/s]

10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 


Training model:   2%|▏         | 1/50 [00:04<03:47,  4.64s/it]

10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 


Training model:   4%|▍         | 2/50 [00:09<03:41,  4.61s/it]

...

10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 


Training model:  98%|█████████▊| 49/50 [03:48<00:04,  4.62s/it]

10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 


Training model: 100%|██████████| 50/50 [03:53<00:00,  4.66s/it]

MSE Mean: 163.35939414348553, MSE Std. Dev: 16.035126259665358


# **C. MSE Mean: 163.359 MSE Std. Dev: 16.035** 

Observations: 
1. Increasing the epochs generated a considerably lower MSE to 163.359 from the previous 349.636
2. This was also able to significantly bring down the standard deviation to 16.03

# **D. Increase the number of hidden layers** 

Repeat part B but use a neural network with the following: 
1. 3 Hidden Layers, 10 Nodes, ReLU 

Compare with step B 

In [21]:
def regression_model(n_cols: int): 
    model = Sequential() 
    model.add(Dense(10, activation='relu', input_shape=(n_cols,))) 
    model.add(Dense(10, activation='relu'))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(1))

    # Compile 
    model.compile(optimizer='adam', loss='mean_squared_error') 
    return model 


def normalize(data: any): 
    mean = data.mean() 
    std = data.std() 
    return (data - mean) / std 

def run_model():  
    # 1. Split 
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size)   
    n_cols = X_train.shape[1] 
    model = regression_model(n_cols) 
    # 1.1 Normalize 
    X_train_normalized = normalize(X_train)
    X_test_normalized = normalize(X_test)
    # 2. Train 
    model.fit(X_train_normalized, y_train, epochs=50, verbose=0) 
    # 3. Evaluate 
    y_predicted = model.predict(X_test_normalized)  
    mse = mean_squared_error(y_test, y_predicted) 
    
    return mse 

In [22]:
from tqdm import tqdm 
# 4. Repeat 50 times 
mse_values = [] 
for _ in tqdm(range(50), desc="Training model"): 
    mse_values.append(run_model()) 

mse_mean = np.mean(mse_values)
mse_std_dev = np.std(mse_values) 
print(f"MSE Mean: {mse_mean}, MSE Std. Dev: {mse_std_dev}")

Training model:   0%|          | 0/50 [00:00<?, ?it/s]

10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 


Training model:   2%|▏         | 1/50 [00:03<02:46,  3.39s/it]

10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 


Training model:   4%|▍         | 2/50 [00:06<02:33,  3.20s/it]

...

10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 


Training model:  98%|█████████▊| 49/50 [02:47<00:03,  3.20s/it]

10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 


Training model: 100%|██████████| 50/50 [02:50<00:00,  3.41s/it]

MSE Mean: 131.99406150301792, MSE Std. Dev: 15.7755440357826


# **D. MSE Mean: 131.994 MSE Std. Dev: 15.775** 

Observations: 
1. Increasing the number of hidden layers exhibits similar behavior to increasing number of epochs: Improved MSE from ~300 to just ~100. 
2. Results also became more consistent

### **Conclusion: Increasing number of layers and number of epochs improves model performance, at the cost of training time and computational resources**